In [1]:
import pandas as pd
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px
import datetime

In [2]:
df = pd.read_csv('media_data.csv')

df.head(15)


,Media,Tactic,Week_start,Year,Quarter,Month,Week,Spending
0,TV - ABC\GMA,TV,2019-12-30,2020,1,1,1,0.0
1,TV - ABC\GMA3,TV,2019-12-30,2020,1,1,1,0.0
2,TV - Today Show,TV,2019-12-30,2020,1,1,1,0.0
3,TV - E!,TV,2019-12-30,2020,1,1,1,0.0
4,TV - Fox News,TV,2019-12-30,2020,1,1,1,0.0
5,TV - Weather Channel,TV,2019-12-30,2020,1,1,1,0.0
6,TV - Recipe.TV,TV,2019-12-30,2020,1,1,1,0.0
7,TV - Simulmedia,TV,2019-12-30,2020,1,1,1,0.0
8,TV - FreeWheel,TV,2019-12-30,2020,1,1,1,0.0
9,TV - Kelly Clarkson Show,TV,2019-12-30,2020,1,1,1,0.0


In [3]:
data1 = df.groupby(['Year', 'Week_start', 'Tactic']).agg({'Spending':'sum'}).reset_index()

In [4]:
data1.head()

,Year,Week_start,Tactic,Spending
0,2016,2015-12-28,DIGITAL,0.0
1,2016,2015-12-28,PRINT,0.0
2,2016,2015-12-28,RADIO,0.0
3,2016,2015-12-28,TV,0.0
4,2016,2015-12-28,VOD,0.0


In [5]:
df['YQ'] = pd.to_datetime(df['Week_start']).dt.to_period('M')

In [6]:
df.head(10)

,Media,Tactic,Week_start,Year,Quarter,Month,Week,Spending,YQ
0,TV - ABC\GMA,TV,2019-12-30,2020,1,1,1,0.0,2019-12
1,TV - ABC\GMA3,TV,2019-12-30,2020,1,1,1,0.0,2019-12
2,TV - Today Show,TV,2019-12-30,2020,1,1,1,0.0,2019-12
3,TV - E!,TV,2019-12-30,2020,1,1,1,0.0,2019-12
4,TV - Fox News,TV,2019-12-30,2020,1,1,1,0.0,2019-12
5,TV - Weather Channel,TV,2019-12-30,2020,1,1,1,0.0,2019-12
6,TV - Recipe.TV,TV,2019-12-30,2020,1,1,1,0.0,2019-12
7,TV - Simulmedia,TV,2019-12-30,2020,1,1,1,0.0,2019-12
8,TV - FreeWheel,TV,2019-12-30,2020,1,1,1,0.0,2019-12
9,TV - Kelly Clarkson Show,TV,2019-12-30,2020,1,1,1,0.0,2019-12


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7384 entries, 0 to 7383
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype    
---  ------      --------------  -----    
 0   Media       7384 non-null   object   
 1   Tactic      7384 non-null   object   
 2   Week_start  7384 non-null   object   
 3   Year        7384 non-null   int64    
 4   Quarter     7384 non-null   int64    
 5   Month       7384 non-null   int64    
 6   Week        7384 non-null   int64    
 7   Spending    7384 non-null   float64  
 8   YQ          7384 non-null   period[M]
dtypes: float64(1), int64(4), object(3), period[M](1)
memory usage: 519.3+ KB


In [8]:
data2 = df.groupby(['Year', 'YQ', 'Tactic']).agg({'Spending':'sum'}).reset_index()

test = data2.copy()

In [9]:
test['YQ'] = test['YQ'].astype(str)
test.index = pd.to_datetime(test['YQ'])

In [10]:
test.head()

,Year,YQ,Tactic,Spending
YQ,,,,
2015-12-01,2016,2015-12,DIGITAL,0.0
2015-12-01,2016,2015-12,PRINT,0.0
2015-12-01,2016,2015-12,RADIO,0.0
2015-12-01,2016,2015-12,TV,0.0
2015-12-01,2016,2015-12,VOD,0.0


In [11]:
pd.to_datetime('2015-09')

Timestamp('2015-09-01 00:00:00')

In [12]:
test['YQ'][0]

'2015-12'

In [13]:
test['Tactic'].sort_values()[330]

'TV'

In [31]:

# Initialize the app
app = JupyterDash(__name__)
app.config.suppress_callback_exceptions = True

def get_options(list_tactics):
    dict_list = []
    for i in list_tactics:
        dict_list.append({'label': i, 'value': i})
    return dict_list

app.layout = html.Div(
    children=[
        html.Div(className='row',
                 children=[
                    html.Div(className='four columns div-user-controls',
                             children=[
                                 html.H2('DASH - KH MEDIA SPENDING'),
                                 html.P('Visualising time series with Plotly - Dash.'),
                                 html.P('Pick one or more media tactic from the dropdown below.'),
                                 html.Div(
                                     className='div-for-dropdown',
                                     children=[
                                         dcc.Dropdown(id='tacticselector', options=get_options(test['Tactic'].unique()),
                                                      multi=True, value=[test['Tactic'].sort_values()[330]],
                                                      className='tacticselector',
                                                      style = dict(#width = '60%',
                                                                    #display = 'inline-block',
                                                                    #verticalAlign = "middle",
                                                                    backgroundColor = 'white')
                                                      ),
                                     ],
                                     style={'color': '#1E1E1E'})
                                ]
                             ),
                    html.Div(className='eight columns div-for-charts bg-grey',
                             children=[
                                 dcc.Graph(id='timeseries', config={'displayModeBar': False}, animate=True)             
                             ])
                              ])
        ])

@app.callback(Output('timeseries', 'figure'),
              [Input('tacticselector', 'value')])
def update_graph(selected_dropdown_value):
    trace1 = []
    df_sub = test.copy()
    # df_sub.index = pd.to_datetime(df_sub['YQ'])
    for tactic in selected_dropdown_value:
        trace1.append(go.Scatter(x=df_sub[df_sub['Tactic'] == tactic].index,
                                 y=df_sub[df_sub['Tactic'] == tactic]['Spending'],
                                 mode='lines',
                                 opacity=0.7,
                                 name=tactic,
                                 textposition='bottom center')) #hovertemplate=None)
        
    xtick  = [pd.to_datetime('2016-03'), pd.to_datetime('2016-06'),pd.to_datetime('2016-09'), pd.to_datetime('2016-12'),
              pd.to_datetime('2017-03'), pd.to_datetime('2017-06'),pd.to_datetime('2017-09'), pd.to_datetime('2017-12'),
              pd.to_datetime('2018-03'), pd.to_datetime('2018-06'),pd.to_datetime('2018-09'), pd.to_datetime('2018-12'),
              pd.to_datetime('2019-03'), pd.to_datetime('2019-06'),pd.to_datetime('2019-09'), pd.to_datetime('2019-12'),
              pd.to_datetime('2020-03'), pd.to_datetime('2020-06'),pd.to_datetime('2020-09'), pd.to_datetime('2020-12')]
    
    ticklabel = ['2016-03','2016-06','2016-09','2016-12',
                 '2017-03','2017-06','2017-09','2017-12',
                 '2018-03','2018-06','2018-09','2018-12',
                 '2019-03','2019-06','2019-09','2019-12',
                 '2020-03','2020-06','2020-09','2020-12']
    
    traces = [trace1]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data,
              'layout': go.Layout(
                  #colorway=["#5E0DAC", '#FF4F00', '#375CB1', '#FF7400', '#FFF400', '#FF0056','cyan', 'skyblue',
                            #'pink', 'tomato'],
                  #template='plotly_dark',
                  paper_bgcolor='#fff',
                  plot_bgcolor='#fff',
                  width=1300,
                  height=500,
                  hovermode = 'closest',
                  margin={'b': 15},
                  autosize=True,
                  title={'text': 'Media Spending by Tactics', 'font': {'color': 'white'}, 'x': 0.5},
                  xaxis={'range': [df_sub.index.min(), df_sub.index.max()], 
                         'showgrid': False, 'tickfont': dict(family='Arial', color = 'black', size=12),
                         'tickvals': xtick, 'ticktext': ticklabel
                        },
                  yaxis={'range': [0, 4250000], 
                         'tickfont': dict(family='Arial', color = 'black', size=12),
                         'tickvals': [i for i in range(500000, 4500000, 500000)],
                         'ticktext': ['$500,000', '$1,000,000', '$1,500,000', 
                                      '$2,000,000', '$2,500,000', '$3,000,000', '$3,500,000', '$4,000,000'],
                         'showline' : False, 
                         'showgrid' : False,
                         'autorange': False},
                  legend=dict(font =dict(family = "Arial", size = 12, color = "black")) 
              ),

              }

    return figure

if __name__ == "__main__":
    app.run_server(debug=True, port=8052)



Dash app running on http://127.0.0.1:8052/
